In [ ]:
# Imports used in this notebook.
import matplotlib as mpl
import ee
import folium
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive
from pprint import pprint as pp
from uuid import uuid1 as uniqueID

# Must authenticate your EE account before use of the package.
ee.Authenticate()
#ee.Initialize()
ee.Initialize(project='ksu-skytruth-vpca-automation', opt_url='https://earthengine-highvolume.googleapis.com')



In [ ]:
import geemap
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).divide(10000)

dateRange = ['2023-04-26', '2023-04-27']


upperLeft = [-81.1583124,  41.0254715 ]
lowerRight = [-81.1469973,  41.0135215 ]
roi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1])
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2023-04-26', '2023-04-27').filterBounds(roi)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 55))
    .map(mask_s2_clouds)
)


  # Water with cloud check, but note this is not bit the addition method.

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(-81.1583124, 41.0254715, 15)
m.add_layer(dataset.mean(), visualization, 'RGB')



/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[41.0254715, -81.1583124], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [ ]:
link = dataset.mean().getDownloadURL({
    'scale': 10,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': roi})
print(link)

https://earthengine-highvolume.googleapis.com/v1/projects/ksu-skytruth-vpca-automation/thumbnails/c8e4f8111825ff8135c0ca4ab4c29ecd-d592226cefca844aca8189c2602ae2eb:getPixels
